Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
#logistic
batch_size = 128
graph = tf.Graph()
#beta_regul = 0.01
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32) #for L2 regularization
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.matmul(tf_train_dataset, weights) + biases
                         
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels= tf_train_labels) )
    loss = tf.reduce_mean(loss + beta_regul * tf.nn.l2_loss(weights) )   
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-2}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 50.708595
Minibatch accuracy: 6.2%
Validation accuracy: 10.7%
Minibatch loss at step 50: 19.108595
Minibatch accuracy: 68.8%
Validation accuracy: 68.8%
Minibatch loss at step 100: 11.342415
Minibatch accuracy: 75.0%
Validation accuracy: 73.8%
Minibatch loss at step 150: 7.424833
Minibatch accuracy: 71.1%
Validation accuracy: 75.3%
Minibatch loss at step 200: 4.363303
Minibatch accuracy: 80.5%
Validation accuracy: 77.2%
Minibatch loss at step 250: 2.862430
Minibatch accuracy: 78.1%
Validation accuracy: 78.1%
Minibatch loss at step 300: 2.114693
Minibatch accuracy: 76.6%
Validation accuracy: 80.6%
Minibatch loss at step 350: 1.730058
Minibatch accuracy: 71.9%
Validation accuracy: 79.8%
Minibatch loss at step 400: 1.091210
Minibatch accuracy: 86.7%
Validation accuracy: 81.8%
Minibatch loss at step 450: 1.129310
Minibatch accuracy: 74.2%
Validation accuracy: 81.3%
Minibatch loss 

#### neural network

In [11]:
batch_size = 128
num_hidden_layer_nodes = 512
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32) #for L2 regularization
    
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    layer1_sum = tf.matmul(tf_train_dataset, weights1) + biases1
    layer1_active = tf.nn.relu(layer1_sum)

    logits = tf.matmul(layer1_active, weights2) + biases2
                         
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels= tf_train_labels) )
    loss = tf.reduce_mean(loss + beta_regul * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))   
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
    

In [12]:
num_steps = 2001
batch_size = 128
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-2}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 1846.655640
Minibatch accuracy: 10.9%
Validation accuracy: 30.6%
Minibatch loss at step 500: 10.943239
Minibatch accuracy: 85.2%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.882715
Minibatch accuracy: 81.2%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.812281
Minibatch accuracy: 78.9%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.831414
Minibatch accuracy: 82.0%
Validation accuracy: 83.7%
Test accuracy: 90.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [13]:
batch_size = 128
num_hidden_layer_nodes = 1024
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32) #for L2 regularization
    
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    layer1_sum = tf.matmul(tf_train_dataset, weights1) + biases1
    layer1_active = tf.nn.relu(layer1_sum)

    logits = tf.matmul(layer1_active, weights2) + biases2
                         
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels) )
    loss = tf.reduce_mean(loss + beta_regul * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))   
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
    

In [17]:
num_steps = 1001
batch_numbers = 2
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = ((step % batch_numbers) * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-2}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
       

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 1927.592896
Minibatch accuracy: 10.9%
Validation accuracy: 29.9%
Minibatch loss at step 250: 128.626221
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 500: 10.620363
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 750: 1.139302
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 0.361576
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Test accuracy: 82.8%


As we can see the accuracy in the minibatch is 100 percent. There are no regularization.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [21]:
batch_size = 128
num_hidden_layer_nodes = 1024
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32) #for L2 regularization
    
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    layer1_sum = tf.matmul(tf_train_dataset, weights1) + biases1
    layer1_active = tf.nn.relu(layer1_sum)
    drop1 = tf.nn.dropout(layer1_active, 0.5)
    logits = tf.matmul(drop1, weights2) + biases2
                         
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels= tf_train_labels) )
    loss = tf.reduce_mean(loss + beta_regul * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))   
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
    

In [23]:
num_steps = 1001
num_batches = 4
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-2}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 150 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
       

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3632.438721
Minibatch accuracy: 8.6%
Validation accuracy: 26.7%
Minibatch loss at step 150: 701.879822
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 300: 156.153946
Minibatch accuracy: 98.4%
Validation accuracy: 76.8%
Minibatch loss at step 450: 34.777180
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 600: 7.984911
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 750: 2.111103
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 900: 0.769007
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 84.6%


This model is a little better.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [62]:
batch_size = 128
num_hidden_layer_nodes1 = 1024
num_hidden_nodes2 = 512
num_hidden_nodes3 = 256

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    global_step = tf.Variable(0)
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)
    
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer_nodes1]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_layer_nodes1)))
    biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    
    weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
    biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
    
    weights4 = tf.Variable(tf.truncated_normal([num_hidden_nodes3, num_labels],  stddev=np.sqrt(2.0 / num_hidden_nodes3)))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
    
    
    layer1_sum = tf.matmul(tf_train_dataset, weights1) + biases1
    layer1_active = tf.nn.relu(layer1_sum)
    drop1 = tf.nn.dropout(layer1_active, 0.5)
    layer2_active = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
    drop2 = tf.nn.dropout(layer2_active, 0.5)
    layer3_active = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
    
    logits = tf.matmul(layer3_active, weights4) + biases4
                         
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels= tf_train_labels))
    loss = tf.reduce_mean(loss) + beta_regul * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))   
    
    
    learning_rate = tf.train.exponential_decay(0.5, global_step, 3000, 0.6, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,  global_step=global_step)

    train_prediction = tf.nn.softmax(logits)
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    layer2_valid = tf.nn.relu(tf.matmul(layer1_valid, weights2) + biases2)
    layer3_valid = tf.nn.relu(tf.matmul(layer2_valid, weights3) + biases3)
    valid_prediction = tf.nn.softmax(tf.matmul(layer3_valid, weights4) + biases4)
    
    
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    layer2_test = tf.nn.relu(tf.matmul(layer1_test, weights2) + biases2)
    layer3_test = tf.nn.relu(tf.matmul(layer2_test, weights3) + biases3)
    test_prediction = tf.nn.softmax(tf.matmul(layer3_test, weights4) + biases4)

In [64]:
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 4e-3}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 8.070715
Minibatch accuracy: 10.2%
Validation accuracy: 16.7%
Minibatch loss at step 500: 1.503081
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.881127
Minibatch accuracy: 80.5%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 0.814624
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Minibatch loss at step 2000: 0.886061
Minibatch accuracy: 82.0%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 0.723923
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 0.904511
Minibatch accuracy: 77.3%
Validation accuracy: 85.6%
Minibatch loss at step 3500: 0.606214
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 4000: 0.694470
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Minibatch loss at step 4500: 0.712153
Minibatch accuracy: 82.8%
Validation accuracy: 86.2%
Minibatc